<a href="https://colab.research.google.com/github/cepdnaclk/e19-co544-Bitcoin-Transaction-Analysis-for-Ransomware-Identification/blob/main/deployments.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
%%capture
!pip install mlflow
!pip install pyngrok

In [ ]:
import mlflow
import subprocess
from pyngrok import ngrok, conf
import getpass

In [ ]:
MLFLOW_TRACKING_URI = "sqlite:///mlflow.db"
subprocess.Popen(["mlflow", "ui", "--backend-store-uri", MLFLOW_TRACKING_URI])

<Popen: returncode: None args: ['mlflow', 'ui', '--backend-store-uri', 'sqli...>

In [ ]:
mlflow.set_tracking_uri(MLFLOW_TRACKING_URI)
# mlflow will create an experiment if it doesn't exist
mlflow.set_experiment("duration-prediction-experiment")

<Experiment: artifact_location='/content/mlruns/1', creation_time=1719404768899, experiment_id='1', last_update_time=1719404768899, lifecycle_stage='active', name='duration-prediction-experiment', tags={}>

In [ ]:
print("Enter your authtoken, which can be copied from https://dashboard.ngrok.com/auth")
conf.get_default().auth_token = getpass.getpass()
port=5000
public_url = ngrok.connect(port).public_url
print(f' * ngrok tunnel \"{public_url}\" -> \"http://127.0.0.1:{port}\"')

Enter your authtoken, which can be copied from https://dashboard.ngrok.com/auth
··········
 * ngrok tunnel "https://daf8-34-106-119-54.ngrok-free.app" -> "http://127.0.0.1:5000"


In [ ]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report, accuracy_score
from sklearn.metrics import accuracy_score, classification_report, precision_score, recall_score, f1_score
from sklearn.preprocessing import PolynomialFeatures
import mlflow
import mlflow.sklearn
from google.colab import drive
from sklearn.svm import SVC
from sklearn.ensemble import RandomForestClassifier
import mlflow.xgboost
import xgboost as xgb

In [ ]:
# Mount Google Drive

from google.colab import drive
drive.mount('/content/drive/')

ValueError: mount failed

In [ ]:
# Load datasets
test_set = pd.read_csv('/content/drive/MyDrive/dataset_project/bitcoin_test_dataset.csv')
training_set = pd.read_csv('/content/drive/MyDrive/dataset_project/bitcoin_training_dataset.csv')

# Prepare data
X_train = training_set.drop(columns=['label'])
X_test = test_set.drop(columns=['label'])
y_train = training_set['label']
y_test = test_set['label']


model 1

In [ ]:
# Create polynomial features
poly = PolynomialFeatures(degree=3)
X_poly_train = poly.fit_transform(X_train)
X_poly_test = poly.transform(X_test)

# Set the experiment name (create if it doesn't exist)
experiment_name = "Bitcoin_Prediction_Experiment"
mlflow.set_experiment(experiment_name)

# Start a new MLflow run with a descriptive run name
with mlflow.start_run(run_name="Initial_Logistic_Regression_Training"):
    # Train the model
    model = LogisticRegression(max_iter=4000)
    model.fit(X_poly_train, y_train)
    y_pred = model.predict(X_poly_test)

    # Evaluate the model
    accuracy = accuracy_score(y_test, y_pred)
    precision = precision_score(y_test, y_pred, average='macro')
    recall = recall_score(y_test, y_pred, average='macro')
    f1 = f1_score(y_test, y_pred, average='macro')
    report = classification_report(y_test, y_pred)

    # Log the model
    mlflow.sklearn.log_model(model, "logistic_regression_model")

    # Register the model
    model_uri = "runs:/{}/logistic_regression_model".format(mlflow.active_run().info.run_id)
    mlflow.register_model(model_uri=model_uri, name="logistic_regression_model")

    # Log parameters and metrics
    mlflow.log_param("solver", "lbfgs")
    mlflow.log_param("max_iter", model.max_iter)
    mlflow.log_metric("accuracy", accuracy)
    mlflow.log_metric("precision", precision)
    mlflow.log_metric("recall", recall)
    mlflow.log_metric("f1", f1)

    # Log classification report as an artifact
    with open("classification_report.txt", "w") as f:
        f.write(report)
    mlflow.log_artifact("classification_report.txt")

    print(f"Initial Accuracy: {accuracy}")
    print("Initial Classification Report:")
    print(report)

model 2

In [ ]:

# Start a new MLflow run with a descriptive run name
with mlflow.start_run(run_name="SVM_Training"):
    # Train the model
    model = SVC()
    model.fit(X_train, y_train)

    # Prediction
    y_train_pred = model.predict(X_train)
    y_pred = model.predict(X_test)

    # Classification report
    train_report = classification_report(y_train, y_train_pred)
    test_report = classification_report(y_test, y_pred)

    # Evaluate the model
    train_accuracy = accuracy_score(y_train, y_train_pred)
    test_accuracy = accuracy_score(y_test, y_pred)

    print("Training classification report:\n", train_report)
    print("Test classification report:\n", test_report)

    # Log the model
    mlflow.sklearn.log_model(model, "svm_model")

    # Register the model (optional)
    model_uri = "runs:/{}/svm_model".format(mlflow.active_run().info.run_id)
    mlflow.register_model(model_uri=model_uri, name="svm_model")

    # Log parameters and metrics
    mlflow.log_param("kernel", model.kernel)
    mlflow.log_param("C", model.C)
    mlflow.log_param("degree", model.degree)
    mlflow.log_param("gamma", model.gamma)


    mlflow.log_metric('accuracy', accuracy)
    mlflow.log_metric('precision', precision)
    mlflow.log_metric('recall', recall)
    mlflow.log_metric('f1', f1)

    # Log classification reports as artifacts
    with open("train_classification_report.txt", "w") as f:
        f.write(train_report)
    mlflow.log_artifact("train_classification_report.txt")

    with open("test_classification_report.txt", "w") as f:
        f.write(test_report)
    mlflow.log_artifact("test_classification_report.txt")


model 3

In [ ]:


best_params = {'bootstrap': True, 'max_depth': 30, 'max_features': 'sqrt', 'min_samples_leaf': 4, 'min_samples_split': 10, 'n_estimators': 200}

# Start a new MLflow run with a descriptive run name
with mlflow.start_run(run_name="Random_Forest_Training"):
    # Create a Random Forest Classifier
    rf = RandomForestClassifier(**best_params)

    # Fit the model
    rf.fit(X_train, y_train)

    # Predict the test set
    y_pred = rf.predict(X_test)

    # Evaluate the model
    accuracy = accuracy_score(y_test, y_pred)
    precision = precision_score(y_test, y_pred)
    recall = recall_score(y_test, y_pred)
    f1 = f1_score(y_test, y_pred)

    print("Accuracy: ", accuracy)
    print("Precision: ", precision)
    print("Recall: ", recall)
    print("F1: ", f1)

    # Classification Report
    class_report = classification_report(y_test, y_pred)
    print(class_report)

    # Log the model
    mlflow.sklearn.log_model(rf, "random_forest_model")

    # Register the model (optional)
    model_uri = "runs:/{}/random_forest_model".format(mlflow.active_run().info.run_id)
    mlflow.register_model(model_uri=model_uri, name="random_forest_model")

    # Log parameters
    mlflow.log_params(best_params)

    # Log metrics
    mlflow.log_metric("accuracy", accuracy)
    mlflow.log_metric("precision", precision)
    mlflow.log_metric("recall", recall)
    mlflow.log_metric("f1", f1)

    # Log classification report as an artifact
    with open("classification_report.txt", "w") as f:
        f.write(class_report)
    mlflow.log_artifact("classification_report.txt")

model 4


In [ ]:
import mlflow
import mlflow.xgboost
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, classification_report
import xgboost as xgb
import pandas as pd

# Load the dataset
bitcoin_dataset_oversampled = pd.read_csv('/content/drive/MyDrive/BitcoinStandardScalerOversampled.csv')

# Split the data into features (X) and target (y)
target = 'label'
X = bitcoin_dataset_oversampled.drop(columns=[target])
y = bitcoin_dataset_oversampled[target]

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

def evaluate_model(y_true, y_pred, model_name="Model"):
    precision = precision_score(y_true, y_pred, average='macro')
    recall = recall_score(y_true, y_pred, average='macro')
    f1 = f1_score(y_true, y_pred, average='macro')
    accuracy = accuracy_score(y_true, y_pred)

    # Log metrics
    mlflow.log_metric('accuracy', accuracy)
    mlflow.log_metric('precision', precision)
    mlflow.log_metric('recall', recall)
    mlflow.log_metric('f1', f1)

    print(f"--- {model_name} ---")
    print(f"Accuracy: {accuracy:.3f}")
    print(f"Precision: {precision:.2f}")
    print(f"Recall: {recall:.2f}")
    print(f"F1 Score: {f1:.2f}")
    print("\n")
    return accuracy, precision, recall, f1

with mlflow.start_run(run_name="XGBoost_Training"):
    # Convert Data to DMatrix
    dtrain = xgb.DMatrix(X_train, label=y_train)
    dtest = xgb.DMatrix(X_test, label=y_test)

    # Set Parameters
    params = {
        'booster': 'gbtree',
        'objective': 'multi:softprob',
        'num_class': 3,
        'eta': 0.3,
        'max_depth': 6,
        'eval_metric': 'mlogloss'
    }

    # Train the Model
    num_round = 100
    bst = xgb.train(params, dtrain, num_round)

    # Predict the test set
    preds = bst.predict(dtest)
    best_preds = [int(np.argmax(line)) for line in preds]

    # Log the model
    mlflow.xgboost.log_model(bst, 'model')

    # Evaluate the model and log the evaluation
    accuracy, precision, recall, f1 = evaluate_model(y_test, best_preds, "XGBoost")

    # Log parameters
    mlflow.log_params(params)

    # Log classification report as an artifact
    class_report = classification_report(y_test, best_preds)
    with open("classification_report.txt", "w") as f:
        f.write(class_report)
    mlflow.log_artifact("classification_report.txt")

    # Register the Model
    model_name = 'xgboost_m1'
    model_uri = 'runs:/{}/model'.format(mlflow.active_run().info.run_id)
    mlflow.register_model(model_uri=model_uri, name=model_name)


predictions

In [ ]:
import mlflow.pyfunc

# Load the model using the registered model name and version
model_name = "logistic_regression_model"
model_version = 1  # Specify the version of the model you want to use
model = mlflow.pyfunc.load_model(model_uri=f"models:/{model_name}/{model_version}")

# Prepare a new data point for prediction (replace with actual data)
new_data = pd.DataFrame({
    "length": [1.080547],
    "weight": [-1.245423],
    "neighbors": [2.209168],
    "income": [1.838642],
    "looped": [1.763154],
    "count": [0.223435]
})

poly = PolynomialFeatures(degree=3)  # You can change the degree as needed
new_data_poly = poly.fit_transform(new_data)


# Make predictions
predictions = model.predict(new_data_poly)
print("Predictions:", predictions)


In [ ]:
import mlflow.pyfunc

# Load the model using the registered model name and version
model_name = "svm_model"
model_version = 1  # Specify the version of the model you want to use
model = mlflow.pyfunc.load_model(model_uri=f"models:/{model_name}/{model_version}")

# Prepare a new data point for prediction (replace with actual data)
new_data = pd.DataFrame({
    "length": [1.080547],
    "weight": [-1.245423],
    "neighbors": [2.209168],
    "income": [1.838642],
    "looped": [1.763154],
    "count": [0.223435]
})

# Make predictions
predictions = model.predict(new_data)
print("Predictions:", predictions)


In [ ]:
import mlflow.pyfunc

# Load the model using the registered model name and version
model_name = "random_forest_model"
model_version = 1  # Specify the version of the model you want to use
model = mlflow.pyfunc.load_model(model_uri=f"models:/{model_name}/{model_version}")

# Prepare a new data point for prediction (replace with actual data)
new_data = pd.DataFrame({
    "length": [1.080547],
    "weight": [-1.245423],
    "neighbors": [2.209168],
    "income": [1.838642],
    "looped": [1.763154],
    "count": [0.223435]
})

# Make predictions
predictions = model.predict(new_data)
print("Predictions:", predictions)

In [ ]:
import mlflow.pyfunc

# Load the model using the registered model name and version
model_name = "xgboost_m1"
model_version = 1  # Specify the version of the model you want to use
model = mlflow.pyfunc.load_model(model_uri=f"models:/{model_name}/{model_version}")

# Prepare a new data point for prediction (replace with actual data)
new_data = pd.DataFrame({
    "length": [1.080547],
    "weight": [-1.245423],
    "neighbors": [2.209168],
    "income": [1.838642],
    "looped": [1.763154],
    "count": [0.223435]
})

# Make predictions
predictions = model.predict(new_data)
print("Predictions:", predictions[0][0])